In [1]:
import pandas as pd
import numpy as np
import os
import glob
from pprint import pprint

In [2]:
# List of state abbreviations
state_abbreviations = ['AZ', 'CA', 'CO', 'ID', 'MT', 'NM', 'NV', 'OR', 'UT', 'WA', 'WY']

bea_df = pd.read_csv('/Users/natebender/Documents/ArcGIS/Projects/Western_protected_lands/BEA_economics_CAEMP25N_clean/BEA_combined_2001_2021.csv')
census_df = pd.read_csv('/Users/natebender/Documents/ArcGIS/Projects/Western_protected_lands/census_clean/census_2001_2021_clean.csv')

In [3]:
print(len(bea_df))
print(len(census_df))

414
414


In [4]:
bea_df.head(2)

,geofips,linecode,description,yr_2001,yr_2002,yr_2003,yr_2004,yr_2005,yr_2006,yr_2007,...,yr_2018,yr_2019,yr_2020,yr_2021,region,division,state,county,stname,ctyname
0,4001,10.0,total employment,23502,23967,24731,25532,26469,27369,29485,...,28078,27790,27089,27051,4.0,8.0,4.0,1.0,Arizona,Apache County
1,4003,10.0,total employment,50753,50600,52389,53549,55625,56566,58186,...,51342,51663,52695,52429,4.0,8.0,4.0,3.0,Arizona,Cochise County


In [5]:
census_df.head(2)

,region,division,state,county,stname,ctyname,geofips,yr_2001,yr_2002,yr_2003,...,yr_2014,yr_2015,yr_2016,yr_2017,yr_2018,yr_2019,yr_2020,yr_2021,description,linecode
0,4,8,4,1,Arizona,Apache County,4001,67863,67319,68072,...,71808,71026,71418,71538,71819,71920,65911,65385,population,20
1,4,8,4,3,Arizona,Cochise County,4003,118798,119847,120638,...,127437,126594,125885,125048,126581,127248,125522,125763,population,20


In [6]:
merge_df = pd.concat([census_df, bea_df], ignore_index=True)

In [21]:
merge_df

,region,division,state,county,stname,ctyname,geofips,yr_2001,yr_2002,yr_2003,...,yr_2014,yr_2015,yr_2016,yr_2017,yr_2018,yr_2019,yr_2020,yr_2021,description,linecode
0,4.0,8.0,4.0,1.0,Arizona,Apache County,4001,67863,67319,68072,...,71808,71026,71418,71538,71819,71920,65911,65385,population,20.0
1,4.0,8.0,4.0,3.0,Arizona,Cochise County,4003,118798,119847,120638,...,127437,126594,125885,125048,126581,127248,125522,125763,population,20.0
2,4.0,8.0,4.0,5.0,Arizona,Coconino County,4005,118283,121308,122882,...,137675,139100,140609,141300,142914,143967,145180,142780,population,20.0
3,4.0,8.0,4.0,7.0,Arizona,Gila County,4007,51274,51478,51337,...,53074,53020,53400,53619,53833,54077,53351,53521,population,20.0
4,4.0,8.0,4.0,9.0,Arizona,Graham County,4009,33356,33224,32985,...,38123,37888,37859,37530,38032,38890,38638,38882,population,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823,4.0,8.0,56.0,37.0,Wyoming,Sweetwater County,56037,24193,23649,24550,...,29976,29133,27833,27866,27920,27915,25683,25444,total employment,10.0
824,4.0,8.0,56.0,39.0,Wyoming,Teton County,56039,23232,23136,23137,...,29898,30711,32068,32744,33631,34019,33081,35628,total employment,10.0
825,4.0,8.0,56.0,41.0,Wyoming,Uinta County,56041,11708,12050,11833,...,12353,12210,11789,11699,11812,11860,11488,11554,total employment,10.0
826,4.0,8.0,56.0,43.0,Wyoming,Washakie County,56043,5232,5148,5157,...,5389,5394,5250,5302,5254,5092,5079,5174,total employment,10.0


In [28]:
new_rows = []  # Step 1: Create an empty list for new rows

# Step 2: Iterate through unique 'geofips'
for geofips in merge_df['geofips'].unique():
    # Extract data for the current 'geofips'
    subset = merge_df[merge_df['geofips'] == geofips]

    # Calculate 'jobs_per_100k' for each year
    for year in range(2001, 2022):
        column = f'yr_{year}'
        linecode_jobs = 10.0
        linecode_population = 20.0
        
        #
        # WHY IS THE JOBS AND POPULATION DATA NOT UPDATING??????????
        #
        
        
        # Calculate 'jobs_per_100k' for this year
        jobs_data = subset[subset['linecode'] == linecode_jobs][column].values[0]
        population_data = subset[subset['linecode'] == linecode_population][column].values[0]
        
        jobs_per_100k = (jobs_data / population_data) * 100000

        # Create a temporary DataFrame for each iteration
        temp_df = pd.DataFrame({
            'geofips': geofips,
            'year': range(2001, 2022),
            'jobs_data': jobs_data,
            'population_data': population_data,
            'jobs_per_100k': jobs_per_100k
        })

        # Print the temporary DataFrame for each iteration
        print(f"Temporary DataFrame for geofips {geofips}:")
        print(temp_df)
        break
        
        # Create a new row with the calculated values
        new_row = subset.iloc[0].copy()
        new_row['description'] = 'jobs_per_100k'
        new_row['linecode'] = 30.0
        new_row[jobs_column] = jobs_per_100k  # Set the jobs_per_100k value for this year
        
        # Append the new row to the list
        new_rows.append(new_row)


        


Temporary DataFrame for geofips 4001:
    geofips  year  jobs_data  population_data  jobs_per_100k
0      4001  2001      23502            67863   34631.537067
1      4001  2002      23502            67863   34631.537067
2      4001  2003      23502            67863   34631.537067
3      4001  2004      23502            67863   34631.537067
4      4001  2005      23502            67863   34631.537067
5      4001  2006      23502            67863   34631.537067
6      4001  2007      23502            67863   34631.537067
7      4001  2008      23502            67863   34631.537067
8      4001  2009      23502            67863   34631.537067
9      4001  2010      23502            67863   34631.537067
10     4001  2011      23502            67863   34631.537067
11     4001  2012      23502            67863   34631.537067
12     4001  2013      23502            67863   34631.537067
13     4001  2014      23502            67863   34631.537067
14     4001  2015      23502            67863  

    geofips  year  jobs_data  population_data  jobs_per_100k
0     53019  2001       2611             7285   35840.768703
1     53019  2002       2611             7285   35840.768703
2     53019  2003       2611             7285   35840.768703
3     53019  2004       2611             7285   35840.768703
4     53019  2005       2611             7285   35840.768703
5     53019  2006       2611             7285   35840.768703
6     53019  2007       2611             7285   35840.768703
7     53019  2008       2611             7285   35840.768703
8     53019  2009       2611             7285   35840.768703
9     53019  2010       2611             7285   35840.768703
10    53019  2011       2611             7285   35840.768703
11    53019  2012       2611             7285   35840.768703
12    53019  2013       2611             7285   35840.768703
13    53019  2014       2611             7285   35840.768703
14    53019  2015       2611             7285   35840.768703
15    53019  2016       

In [24]:
len(new_rows)

8694

In [ ]:
# Concatenate the original merge_df with the result_df
        # Step 6: Concatenate the new rows with the original 'merge_df'
results = pd.concat([merge_df] + new_rows, ignore_index=True)

# If needed, sort the DataFrame by 'geofips' or other columns
# merge_df = merge_df.sort_values(by=['geofips'])

# Now merge_df contains the calculated 'jobs_per_100k' values
